# Data Cleaning 

#### 1. Import pandas library.

In [5]:
import pandas as pd
import sys
!{sys.executable} -m pip install pymysql

  Using cached https://files.pythonhosted.org/packages/ed/39/15045ae46f2a123019aa968dfcba0396c161c20f855f11dea6796bcaae95/PyMySQL-0.9.3-py2.py3-none-any.whl


#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


In [7]:
from sqlalchemy import create_engine
import pymysql

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

In [8]:
driver = 'mysql+pymysql:'
user = 'guest'
password = 'relational'
ip = 'relational.fit.cvut.cz'
database = 'stats'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

#### 4. Import the users table.

In [12]:
query = """
        SELECT * FROM users
"""
df_users = pd.read_sql(query, engine)
df_users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 5. Rename Id column to userId.

In [13]:
df_users = df_users.rename(columns={'Id': 'userId'})
df_users.head()

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 6. Import the posts table. 

In [75]:
query = """
        SELECT * FROM posts
"""
df_posts = pd.read_sql(query, engine)
df_posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None


#### 7. Rename Id column to postId and OwnerUserId to userId.

In [80]:
df_posts = df_posts.rename(columns={'Id': 'postId', 'OwnerUserId':'userId'})
df_posts.head()


Index(['postId', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'userId', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName'],
      dtype='object')

#### 8. Define new dataframes for users and posts with the following selected columns:
**users columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts columns**: postId, Score, userID, ViewCount, CommentCount

In [83]:
df_new = pd.DataFrame(df_users[:])

lst = ['CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe',
       'AccountId', 'Age', 'ProfileImageUrl']

for key in lst:
    df_new = df_new.drop(key, axis=1)

df_new.head()

lst =['PostTypeId', 'AcceptedAnswerId', 'CreaionDate',
     'Body', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName']
df_new = pd.concat([df_new, df_posts], axis=1)

for key in lst:
    df_new = df_new.drop(key, axis=1)
    
df_new.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,userId,CommentCount
0,-1.0,1.0,0.0,5007.0,1920.0,1,23,1278.0,8.0,1
1,2.0,101.0,25.0,3.0,0.0,2,22,8198.0,24.0,1
2,3.0,101.0,22.0,19.0,0.0,3,54,3613.0,18.0,4
3,4.0,101.0,11.0,0.0,0.0,4,13,5224.0,23.0,2
4,5.0,6792.0,1145.0,662.0,5.0,5,81,NaN,23.0,3


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [89]:
df_merged = df_users.merge(df_posts)

In [90]:
df_merged.columns

Index(['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl', 'postId', 'PostTypeId',
       'AcceptedAnswerId', 'CreaionDate', 'Score', 'ViewCount', 'Body',
       'LasActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'LastEditorUserId', 'LastEditDate',
       'CommunityOwnedDate', 'ParentId', 'ClosedDate', 'OwnerDisplayName',
       'LastEditorDisplayName'],
      dtype='object')

#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [92]:
df_merged.apply(lambda x: x.count(), axis=1).sum()


1951393

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [93]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90584 entries, 0 to 90583
Data columns (total 34 columns):
userId                   90584 non-null int64
Reputation               90584 non-null int64
CreationDate             90584 non-null datetime64[ns]
DisplayName              90584 non-null object
LastAccessDate           90584 non-null datetime64[ns]
WebsiteUrl               32071 non-null object
Location                 40452 non-null object
AboutMe                  41684 non-null object
Views                    90584 non-null int64
UpVotes                  90584 non-null int64
DownVotes                90584 non-null int64
AccountId                90584 non-null int64
Age                      26804 non-null float64
ProfileImageUrl          24063 non-null object
postId                   90584 non-null int64
PostTypeId               90584 non-null int64
AcceptedAnswerId         14563 non-null float64
CreaionDate              90584 non-null datetime64[ns]
Score                    90

In [95]:
convert_dict = {'Age': int, 
               } 
df_merged = df_merged.astype(convert_dict)
print(df_merged.info())

ValueError: Cannot convert non-finite values (NA or inf) to integer